In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR

In [47]:
# reading the data
df = pd.read_csv(r"..\data\AAPL Stock Price (15-06-204 and Last 30 days).csv", 
                 parse_dates=['Date'], dayfirst=True, index_col='Date')
df.columns = df.columns.str.lower()
df.index = df.index.strftime('%Y-%m-%d')

In [48]:
display(df.head(),df.tail(),df.sample(5))

,open,high,low,close,volume
Date,,,,,
2024-05-16,190.47,191.10,189.66,189.84,52845230
2024-05-17,189.51,190.81,189.18,189.87,41282925
2024-05-20,189.35,191.92,189.01,191.04,44361275
2024-05-21,191.09,192.73,190.92,192.35,42309401
2024-05-22,192.27,192.82,190.27,190.90,34648547


,open,high,low,close,volume
Date,,,,,
2024-06-10,196.90,197.30,192.15,193.12,97262077
2024-06-11,193.65,207.16,193.63,207.15,172373296
2024-06-12,207.37,220.20,206.90,213.07,198134293
2024-06-13,214.74,216.75,211.60,214.24,97862729
2024-06-14,213.85,215.17,211.30,212.49,70122748


,open,high,low,close,volume
Date,,,,,
2024-05-20,189.35,191.92,189.01,191.04,44361275
2024-06-03,192.90,194.99,192.52,194.03,50080539
2024-05-23,190.98,191.00,186.63,186.88,51005924
2024-06-13,214.74,216.75,211.60,214.24,97862729
2024-06-12,207.37,220.20,206.90,213.07,198134293


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 2024-05-16 to 2024-06-14
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    21 non-null     float64
 1   high    21 non-null     float64
 2   low     21 non-null     float64
 3   close   21 non-null     float64
 4   volume  21 non-null     int64  
dtypes: float64(4), int64(1)
memory usage: 1008.0+ bytes
